In [1]:
import os
import sys

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
import logging
from datetime import datetime as dt


now = dt.today()
logging.basicConfig(filename=f'{now}.debug.txt', level=logging.CRITICAL)

In [4]:
username="your_username"
password="your_password"

In [5]:
driver = webdriver.Firefox()
geektime_login = 'https://account.geekbang.org/login'
driver.get(geektime_login)

input_phone = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[1]/input")
input_pass = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/input")

input_phone.send_keys(username)
input_pass.send_keys(password)

In [6]:
# 登录 
button_submit = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/button")
button_submit.click()

In [7]:
def logout():
    logout_node = None
    try:
        logout_node = driver.find_element_by_css_selector("div[class~='_1MA-AdXE_0'] a:last-child")
    except:
        try:
            logout_node = driver.find_element_by_css_selector("a.avatarwrap")
        except:
            pass
    
    if logout_node is not None:
        logout_node.click()
    else:
        account_node = None
        try:
            account_node = driver.find_element_by_css_selector("span[class~='iconfont']")
            account_node.click()
            logout_node = driver.find_element_by_css_selector("div[class~='_1MA-AdXE_0'] a:last-child")
        except Exception as e1:
            try:
                account_node = driver.find_element_by_css_selector("a.avatarwrap")
                account_node.click()
                logout_node = driver\
                            .find_element_by_css_selector("div[class~='profile-dropdown'] a:last-child")
            except Exception as e2: 
                logging.error("The account node cannot be found!")
                print(e1)
                print(e2)
        else:
            logout_node.click()
    try:
        WebDriverWait(driver, 10, 1).until(lambda d: d.find_element_by_css_selector("div#id"))
    except Exception as e:
        print(e, file=sys.stderr)
    finally:
        driver.quit()


In [8]:
# 等待登陆成功 并进入我的基本信息页
WebDriverWait(driver, 20, 2).until(lambda x: x.find_elements_by_css_selector("a.navbar-item:nth-child(2)"))

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="bce2ee97-71b1-44fb-8cba-bf2013750748", element="7a38f5f7-7f4a-4a9f-b985-27a6c089f08c")>]

In [9]:
# click 极客时间
geektime_index = "https://time.geekbang.org/"
driver.get(geektime_index)

In [10]:
# 我的课程列表的数据来自的是异步的AJAX请求 故在列表显示之前必须等待
WebDriverWait(driver, 10, 2).until(lambda x: x.find_element_by_xpath("/html/body/div[1]/div[1]/div[5]/div/a"))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="bce2ee97-71b1-44fb-8cba-bf2013750748", element="3893c433-0f60-4605-8f46-08e813238ef9")>

In [11]:
# click 关闭(modal)
driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[5]/div/a").click()

In [12]:
# 点击 我的课程 无法让webdriver的活跃页跟随跳转到新打开的 我的课程 页,
# 直接请求 我的课程 URL 进行跳转
geektime_home = "https://time.geekbang.org/dashboard/course"
driver.get(geektime_home)

In [13]:
# 我的课程列表的数据来自的是异步的AJAX请求 故在列表显示之前必须等待
course_list = None
try:
    course_list = WebDriverWait(driver, 20, 2).until(lambda x: x.find_elements_by_css_selector("button"))
    logging.info(course_list)
except Exception as e:
    print(e, file=sys.stderr)
    logout()

In [14]:
course_home = driver.current_url
course_home

'https://time.geekbang.org/dashboard/course'

In [15]:
# first course
course_list[0].click()

In [16]:
try:
    WebDriverWait(driver, 20, 2).until(lambda x: len(x.window_handles) > 1)
except Exception as e:
    print(e, file=sys.stderr)
    logout()

Click on the button will open a new non-active tab, in other words, `driver.current_url` still remains 'https://time.geekbang.org/dashboard/course' while the course content is on the non-active tab.

In [17]:
driver.current_url

'https://time.geekbang.org/dashboard/course'

In [18]:
driver.window_handles

['15', '4294967338']

In [19]:

## try to close current tab (old tab) to make the course page currentnt tab, but failed for some reason
# driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 'W')

## try to use CTRL + TAB to switch to the right tab, but failed again
#driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.TAB)

# finnaly this one works
driver.switch_to.window(driver.window_handles[1])

In [20]:
# wait unit the chapter container is ready
WebDriverWait(driver, 20, 2)\
    .until(lambda x: x.find_element_by_xpath("//div[contains(@class, 'faLsNCZ2_0')]"))


<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="bce2ee97-71b1-44fb-8cba-bf2013750748", element="732ea95c-18f2-43d8-b7bd-3e27013bf9c8")>

In [21]:
course_url = driver.current_url
course_url

'https://time.geekbang.org/column/article/73971'

In [22]:
chapter_div_list = None
try:
    chapter_div_list = (WebDriverWait(driver, 20, 2)
                        .until(lambda d: d.find_elements_by_css_selector("div[class~='rMLmCpCA_0']")))
except Exception as e:
    print(e, file=sys.stderr)
    driver.refresh()
    (WebDriverWait(driver, 20, 2)
                        .until(lambda d: d.find_elements_by_css_selector("div[class~='rMLmCpCA_0']")))
        

In [23]:
assert len(chapter_div_list) > 0, 'stop'

In [24]:
chapter_title_list = []
for div in chapter_div_list:
    chapter_title_list.append(div.get_property("firstChild").get_property("firstChild")
                              .get_property("innerHTML"))

In [25]:
chapter_title_list

['开篇词 (1讲)',
 '第一模块：数据分析基础篇 (16讲)',
 '第二模块：数据分析算法篇 (20讲)',
 '第三模块：数据分析实战篇 (7讲)',
 '第四模块：数据分析工作篇 (2讲)',
 '加餐 (1讲)',
 '结束语 (1讲)']

In [26]:
def format_path(p):
        # replace "/" with "," due to path format limitation
        p = p.replace(" ", "")
        p = p.replace('/', ',')
        # replace all non-ASCII code with ASCII code
        p = p.replace("丨", "|")
        p = p.replace("（", "(")
        p = p.replace("）",")")
        p = p.replace("：", ":")
        p = p.replace("？", "?")
        p = p.replace("，", ",")
        p = p.replace("“", "\"")
        p = p.replace("”","\"")
        return p


In [27]:
chapter_title_list = list(map(format_path, chapter_title_list))
chapter_title_list

['开篇词(1讲)',
 '第一模块:数据分析基础篇(16讲)',
 '第二模块:数据分析算法篇(20讲)',
 '第三模块:数据分析实战篇(7讲)',
 '第四模块:数据分析工作篇(2讲)',
 '加餐(1讲)',
 '结束语(1讲)']

In [28]:
course_title = driver.find_element_by_css_selector("div._382D6fck_0 > div.WRb7YQxC_0").text
course_title = format_path(course_title)

# create filesystem hierachy
if os.path.isdir(course_title) is False:
    os.mkdir(course_title)
    print(course_title)
    logging.info(course_title)
for chapter in chapter_title_list:
    p = os.path.join(course_title, chapter)
    print(p)
    logging.info(p)
    if os.path.isdir(p) is False:
        os.mkdir(p)


数据分析实战45讲
数据分析实战45讲/开篇词(1讲)
数据分析实战45讲/第一模块:数据分析基础篇(16讲)
数据分析实战45讲/第二模块:数据分析算法篇(20讲)
数据分析实战45讲/第三模块:数据分析实战篇(7讲)
数据分析实战45讲/第四模块:数据分析工作篇(2讲)
数据分析实战45讲/加餐(1讲)
数据分析实战45讲/结束语(1讲)


In [29]:
# reset history and fold all chapter node to get ready for iteration
for chapter in chapter_div_list:
    logging.info(chapter.get_property("className"))
    if "_3kBNzF8S_0" in chapter.get_property("className"):
        chapter.get_property("firstChild").click()

In [30]:
def save_article_to_html():
    course_title = (driver.find_element_by_css_selector("div._382D6fck_0 > div.WRb7YQxC_0")
                    .get_property("innerText"))
    
    current_article_div = driver.find_element_by_css_selector("div[class~='_2AkdpMxn_0']")
    
    current_chapter_div = (current_article_div
                            .get_property("parentElement")
                            .get_property("parentElement")
                            .get_property("parentElement"))
    current_chapter_name = (current_chapter_div
                            .get_property("firstChild")
                            .get_property("firstChild")
                            .get_property("innerText"))
    
    current_article_name = (current_article_div
                            .get_property("firstChild")
                            .get_property("innerText"))
    
    article_content_div = driver.find_element_by_css_selector("div._2c4hPkl9 > div:nth-child(1)")
    article_content_html = article_content_div.get_property("outerHTML")
    
    # write to file
    filename = current_article_name+'.html' 
    
    logging.info(course_title, current_chapter_name, current_article_name)
    output_path = os.path.join(format_path(course_title),
                               # format_path(current_chapter_name),
                               format_path(filename))
    logging.info(output_path)
    
    if os.path.isfile(output_path) is False:
        with open(output_path, 'w') as writer:
            writer.write(article_content_html)
        print("SUCCESS: " + output_path + " is saved.")
    else:
        print("WARNING: " + output_path + " has existed!")

In [31]:
def save_all_article():
    n_chapter = len(chapter_div_list)

    for c in range(n_chapter):
        # first click the chapter div to unfold it
        chapter_div_list[c].get_property("firstChild").location_once_scrolled_into_view
        chapter_div_list[c].get_property("firstChild").click()

        article_node_list = (driver
                             .find_elements_by_css_selector(
                                 "div[class~='_3kBNzF8S_0'] div[class~='_1NHkz9xq_0']")
                            )
        n_article = len(article_node_list)

        for a in range(n_article):
            # click to go to the article
            article_node_list[a].location_once_scrolled_into_view
            article_node_list[a].click()

            # wait for content to be displayed
            try:
                WebDriverWait(driver, 20, 3).until(lambda x: x.find_element_by_css_selector("div._2c4hPkl9"))
            except Exception as e:
                logging.error(article_node_list[a].get_property("innerText")+" failed to save!")
            logging.info(driver.current_url)
            
            try:
                save_article_to_html()
            except Exception as e:
                logging.error(article_node_list[a].get_property("innerText")+" failed to save!")
                print(e, file=sys.stderr)

        current_chapter_div = driver.find_element_by_css_selector("div[class~='_3kBNzF8S_0']")
        current_chapter_div.get_property("firstChild").location_once_scrolled_into_view
        current_chapter_div.get_property("firstChild").click()


In [32]:
save_all_article()

SUCCESS: 数据分析实战45讲/开篇词|你为什么需要数据分析能力?.html is saved.
SUCCESS: 数据分析实战45讲/01|数据分析全景图及修炼指南.html is saved.
SUCCESS: 数据分析实战45讲/02|学习数据挖掘的最佳路径是什么?.html is saved.
SUCCESS: 数据分析实战45讲/03|Python基础语法:开始你的Python之旅.html is saved.
SUCCESS: 数据分析实战45讲/04|Python科学计算:用NumPy快速处理数据.html is saved.
SUCCESS: 数据分析实战45讲/05|Python科学计算:Pandas.html is saved.
SUCCESS: 数据分析实战45讲/06|学数据分析要掌握哪些基本概念?.html is saved.
SUCCESS: 数据分析实战45讲/07|用户画像:标签化就是数据的抽象能力.html is saved.
SUCCESS: 数据分析实战45讲/08|数据采集:如何自动化采集数据?.html is saved.
SUCCESS: 数据分析实战45讲/09|数据采集:如何用八爪鱼采集微博上的"D&G"评论.html is saved.
SUCCESS: 数据分析实战45讲/10|Python爬虫:如何自动化下载王祖贤海报?.html is saved.
SUCCESS: 数据分析实战45讲/11|数据科学家80%时间都花费在了这些清洗任务上?.html is saved.
SUCCESS: 数据分析实战45讲/12|数据集成:这些大号一共20亿粉丝?.html is saved.
SUCCESS: 数据分析实战45讲/13|数据变换:考试成绩要求正态分布合理么?.html is saved.
SUCCESS: 数据分析实战45讲/14|数据可视化:掌握数据领域的万金油技能.html is saved.
SUCCESS: 数据分析实战45讲/15|一次学会Python数据可视化的10种技能.html is saved.
SUCCESS: 数据分析实战45讲/16|数据分析基础篇答疑.html is saved.
SUCCESS: 数据分析实战45讲/17|决策树(上):要不要去打篮球?决策树来告诉你.html 

In [ ]:
logout()